## Practice

1. Input continuous images from 'car.mp4'.
2. For each frame, detect every car using YOLOv8 trained data 'license_plate_detector.pt'. (mark with red rectangles)
3. For each car, detect a licence plate using 'yolov8n.pt'. (mark with blue rectangle)
4. For each licence plate, OCR using Tesseract.
5. Print the recognized licence plate number above each detected licence plate. (putText() in green color).
6. Use whatever you learned this semester to improve the result
7. Upload your .ipynb file.

### Requirements.txt
#### opencv-python==4.9.0.80
#### ultralytics==8.2.15
#### numpy==1.26.4
#### pytesseract==0.3.9

In [6]:
from ultralytics import YOLO
import cv2
import pytesseract
import numpy as np

In [7]:
# Function to draw bounding boxes and text
def draw_annotations(frame, annotations, color, font_scale=0.9):
    for (x1, y1, x2, y2, text) in annotations:
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        if text:
            cv2.putText(frame, text, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), 2)


In [8]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to remove noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply thresholding to get a binary image
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Optionally apply morphology operations to clean up the image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    return binary

In [9]:
# Specify the Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Load models
car_detector = YOLO('yolov8n.pt')
license_plate_detector = YOLO('license_plate_detector.pt')

In [10]:
# Load video
cap = cv2.VideoCapture('car.mp4')

# Define the list of vehicle class IDs (as per your model's class mapping)
vehicles = [2, 3, 5, 7]

# Read frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles
    car_detections = car_detector(frame)[0]
    car_annotations = []
    for detection in car_detections.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = detection
        if int(class_id) in vehicles:
            car_annotations.append((x1, y1, x2, y2, f'Car: {int(score * 100)}%'))

    # Detect license plates
    license_plate_detections = license_plate_detector(frame)[0]
    license_plate_annotations = []
    for license_plate in license_plate_detections.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = license_plate
        license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
        processed_image = preprocess_image(license_plate_crop)
        # Adjust Tesseract OCR configuration for better accuracy
        custom_config = r'--oem 3 --psm 8'
        license_plate_text = pytesseract.image_to_string(processed_image, config=custom_config).strip()
        license_plate_annotations.append((x1, y1, x2, y2, license_plate_text))

    draw_annotations(frame, car_annotations, (0, 0, 255), font_scale = 0.9)  # Red rectangles for cars
    draw_annotations(frame, license_plate_annotations, (255, 0, 0), font_scale = 2)  # Blue rectangles for license plates
    
    frame_resized = cv2.resize(frame, (800, 450))
    
    cv2.imshow('Frame', frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 21 cars, 1 bus, 2 trucks, 90.8ms
Speed: 5.4ms preprocess, 90.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 88.6ms
Speed: 4.0ms preprocess, 88.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 115.2ms
Speed: 5.0ms preprocess, 115.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 83.5ms
Speed: 4.1ms preprocess, 83.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 99.0ms
Speed: 5.2ms preprocess, 99.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 83.5ms
Speed: 2.8ms preprocess, 83.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 97.3ms
Speed: 3.9ms preprocess, 97.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates

## Discussion
According to the result, I try to use Grayscale, Gaussian filter and morphology, the recognition is more better with original solution

## Reference
- [Automatic-Number-Plate-Recognition-YOLOv8](https://github.com/Muhammad-Zeerak-Khan/Automatic-License-Plate-Recognition-using-YOLOv8/tree/main)